# Access to my drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import pickle
import csv
import math
import torch
import keras
import os
import gc
import tensorflow as tf
from nltk.tokenize import word_tokenize
from torch import nn
from tqdm.notebook import tqdm
import torch.nn.functional as F
from keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import sequence
from keras.models import Sequential , Model
from keras.layers import LSTM, Bidirectional, Dense, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from scipy.special import softmax
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import precision_score, recall_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Embedding, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D
import os
from keras.callbacks import ModelCheckpoint, EarlyStopping,Callback,CSVLogger
import gc
class GarbageCollectorCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Reshape, Conv1D, GlobalAveragePooling1D, Dense

# Read data

In [ ]:
def load_from_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data
try:
    with open('/content/drive/MyDrive/ASC/dataset/final_train_bert.pkl', "rb") as f:
        X_final_train_bert = np.load(f, allow_pickle=True)
except IOError:  # Handle potential errors during loading
    X_final_train_bert = []

# Load y_train
y_train = load_from_pickle('/content/drive/MyDrive/ASC/dataset/y_train.pkl')

# Transform data

In [ ]:
from sklearn.preprocessing import OneHotEncoder
basic = [i for i in range(3)]
enc = OneHotEncoder()
enc.fit(np.array(basic).reshape(-1,1))
y_enc = enc.transform(y_train[:39400].values.reshape(-1, 1))
y_enc = [y_enc[j].toarray() for j in range(39400)]

In [ ]:
y_enc = np.array(y_enc).reshape((39400, 3))

# Train model

In [ ]:
def generator(data, labels, batch_size):
  while True:
    indices = np.random.permutation(len(data))
    for i in range(0, len(indices), batch_size):
      batch_indices = indices[i:i + batch_size]
      yield data[batch_indices], labels[batch_indices]

model = Sequential([
    Dense(4, input_shape=(393816,), activation='linear'),
])

for _ in range(15):
    model.add(Dense(units=64, activation='relu'))

model.add(Dense(3, activation='softmax'))


loss_function = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)


model.compile( loss=loss_function, metrics=['accuracy'])


checkpoint_filepath = '/content/drive/MyDrive/ASC/results/models/exp5/'
if not os.path.exists(checkpoint_filepath):
  os.mkdir(checkpoint_filepath)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_filepath, 'z_model_weights_epoch{epoch:02d}_val_acc{val_accuracy:.4f}.h5'),
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
history_logger_1 = CSVLogger(os.path.join(checkpoint_filepath,'history1.csv'), separator=",", append=True)
checkpoint_1 = ModelCheckpoint(checkpoint_filepath, monitor='val_accuracy', verbose=0, save_best_only=True, mode='max')
es_1 = EarlyStopping(monitor='val_loss', patience=5)
callbacks_list_1 = [GarbageCollectorCallback(),model_checkpoint_callback, history_logger_1]

validation_split = 0.1  # You can adjust this based on your needs
num_validation_samples = int(validation_split * len(X_final_train_bert))

X_train = X_final_train_bert[:-num_validation_samples]
y_train = y_enc[:-num_validation_samples]
X_validation = X_final_train_bert[-num_validation_samples:]
y_validation = y_enc[-num_validation_samples:]


batch_size = 256
history = model.fit_generator(
    generator=generator(X_train, y_train, batch_size),
    epochs=1000,
    steps_per_epoch=len(X_train) // batch_size,
    validation_data=generator(X_validation, y_validation, batch_size),  # Use generator for validation data as well
    validation_steps=len(X_validation) // batch_size,
    callbacks=callbacks_list_1
)